# 크롬 확장 프로그램 [[공식문서]](https://support.google.com/chrome/a/answer/2714278?hl=ko)

![4.jpg](images/4.jpg)

크롬 확장 프로그램은 외부에서 제작되어 크롬브라우저를 통해 기능이 실행되는 크롬 브라우저의 기능을 확장하는 프로그램들을 말합니다. 기본적으로 [크롬 웹스토어](https://chrome.google.com/webstore/category/extensions?utm_source=chrome-ntp-icon) 를 통해 다른 개발자들이 만든 프로그램들을 설치해서 사용할 수도 있고 내가 직접 제작해서 나만의 확장된 기능을 만들어 낼 수도 있습니다.

### 크롬 확장 프로그램 구조
크롬 확장 프로그램은 manifest.json, html, js 이렇게 3개의 파일로 구성되어있습니다.
- manifest.json: 기본적인 크롬 확장프로그램의 정보와 권한, 아이콘설정, 기본 html 파일 경로 등을 설정하는 기본 파일입니다.
- html: manifest.json 에 설정된 html 파일이며 기본적으로 확장프로그램 아이콘을 눌렀을때 팝업되는 안내 페이지 정도 입니다.
- js: html 에서 사용되는 js 파일입니다.

### manifest.json [[공식문서]](https://developer.chrome.com/extensions/getstarted)

In [ ]:
%%html

{
    "name": "QR Code 생성기",
    "description": "QR 코드를 만듭니다.",
    "version": "0.0.1",
    "manifest_version": 2,
    "permissions": [
        "tabs", "<all_urls>"
    ],
    
    "browser_action": {
        "default_popup": "qrcode.html"
    },
    "icons": {
        "48": "qrcode.png"
    }
}

manifest.json 의 문법은 json 형태로 작성됩니다.

- name: 크롬 확장 프로그램의 이름을 설정합니다.
- description: 크롬 확장 프로그램의 설명입니다.
- version: 크롬 확장 프로그램의 버전입니다.
- manifest_version: 메니페스트의 버전입니다. 이 버전은 내가 작성하는 프로그램의 버전과 상관없습니다. [공식문서](https://developer.chrome.com/extensions/manifestVersion)
- permissions: 크롬 확장 프로그램의 권한을 설정합니다. [공식문서](https://developer.chrome.com/extensions/declare_permissions)
- browser_action: 크롬 확장프로그램 아이콘을 클릭했을시 처리될 내용입니다.
- icons: 크롬 확장프로그램의 아이콘 파일을 설정합니다.


여기서 중요한 부분이 permissions 인데 예를 들어 내가 작성한 크롬 확장 프로그램이 브라우저의 탭을 열거나 탭의 정보를 필요로 하는 기능이 있는데 권한에 tabs 가 설정되지 않으면 해당 기능은 동작하지 않게 됩니다. 따라서 내가 작성하는 프로그램의 기능에 어떤 권한이 있는지를 모두 명시해야만 정상적으로 동작합니다.

### qrcode.html

In [ ]:
%%html

<html>
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
  <script src="jquery.js"></script>
  <script src="qrcode.js"></script>
</head>
<body style="min-width:150px;-webkit-user-select:none;">
  <div style="text-align:center;">
    <h3>큐알코드 생성기</h3>
  </div>
  <div id="qrcode"></div>
</body>
</html>

크롬 확장프로그램은 반드시 자바스크립트의 영역과 html 영역이 분리 되어있어야 하고 script 링크를 외부에서 가져올 수가 없습니다. 따라서 jquery 등을 사용하려면 반드시 해당 파일을 다운로드하여 로컬에서 연결해야 합니다. 위 파일은 브라우저의 확장프로그램 아이콘을 클릭하면 info.html 파일이 열리는데 jquery.js 와 qrcode.js 가 실행되게 됩니다.

### qrcode.js

In [ ]:
// https://developer.chrome.com/extensions/tabs#method-query

chrome.tabs.query({}, gotTabs);
function gotTabs(tabs) {
    console.log(tabs);
    console.log(tabs[0].url);
    var url = tabs[0].url
    $.ajax({
        url: "http://localhost:9888/qrcode",
        type: "POST",
        cache: false,
        data: {"text": url},
        success: function(result) {
            console.log(result);
            base64 = result.data;
            tag = "<img src='data:image/png;base64, " + base64 + "'>";
            console.log(tag);
            $("#qrcode").append(tag);
        },
        error: function(request, status, error) {
            var msg = "ERROR : " + request.status + "<br>"
            msg +=  + "내용 : " + request.responseText + "<br>" + error;
            console.log(msg);  
        }
    });
}

qrcode.html 이 호출되면 chorom.tabs.query({쿼리}, "함수") 함수를 사용해서 브라우저의 탭에 쿼리를 요청해 정보를 얻어 냅니다. 위에서는 {} 로 탭의 정보를 모두 얻게끔 작성되었습니다. 쿼리 요청의 결과는 "함수" 를 통해 받을 수 있습니다.

### 탭 정보
탭은 복수개일 수 있기 때문에 리스트 형태로 리턴되므로 현재 탭의 정보는 [0] 요소로 접근가능하고 여기서 우리가 필요한 url 정보를 얻어낼 수 있습니다. 나머지 코드는 기존의 코드와 동일한 형태로 서버로 ajax 통신을 하여 해당 URL 데이터의 QR코드를 생성 요청하고 리턴 받아서 qrcode.html 에 있는 div 태그에 출력하는 내용입니다. 여기서 한가지 차이점은 기존의 url_for 함수는 파이썬에서 사용하는 코드였고 이 코드는 독립적으로 동작하는 코드기 때문에 ajax 의 url 값에 url_for 함수를 사용할 수 없으므로 직접 주소를 작성해야 한다는 점입니다. 만약 QR코드를 작성하는 파이썬 코드를 독립적인 서버로 운영한다면 해당 서버의 URL 주소를 기재해야 합니다.

### 확장프로그램 등록
![6.jpg](images/6.jpg)


크롬 브라우저의 메뉴 > 도구 더보기 > 확장 프로그램
<br><br>


![7.jpg](images/7.jpg)


개발자모드를 체크 하시고 압축해제된 확장 프로그램을 로드합니다를 클릭하여 크롬 확장프로그램이 작성된 폴더를 선택하면 됩니다.
<br><br>


### 완성된 모습

이제 크롬으로 접속중인 브라우저의 URL을 QR코드로 즉시 생성하여 스마트폰 등을 통해 해당 페이지를 접속하거나 할 수 있습니다.

![5.jpg](images/5.jpg)